In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 2 : experimentation: get started with Vertex Tensorboard

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_tensorboard.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 2 : experimentation: get started with Vertex Tensorboard.

### Objective

In this tutorial, you learn how to use `Vertex AI TensorBoard` when training with `Vertex AI`.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI TensorBoard`
- `Vertex AI Training`

The steps performed include:

- Create a TensorBoard callback when training a model.
- Using Tensorboard with locally trained model.
- Using Vertex AI TensorBoard with Vertex AI Training.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.


### Recommendations

When doing E2E MLOps on Google Cloud, the following are the best practices for visualizing your training with TensorBoard.

#### Local TensorBoard

Use the OSS version of TensorBoard, either command-line or daemon version, when doing ad-hoc training locally.

#### Cloud TensorBoard

Use the Tensorboard.dev, when doing training on the cloud -- unless you have a privacy issue.

#### Experiments

Use Vertex AI TensorBoard when you have a privacy issue or doing experiments to compare results for different experiment configurations.

## Installations

Install *one time* the packages for executing the MLOps notebooks.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip3 install -U tensorflow==2.5 $USER_FLAG
! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG

     |████████████████████████████████| 454.4 MB 12 kB/s s eta 0:00:01��███████████▍| 445.2 MB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 32.6 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 28.4 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.2 MB 34.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 36.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 45.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 14.9 MB 46.8 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 45.2 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 57.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 35.8 MB/s 

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: vertex-ai-dev


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



To take a quick anonymous survey, run:
  $ gcloud survey



#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [6]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [7]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [8]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://vertex-ai-devaip-20220401072400/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [9]:
! gsutil ls -al $BUCKET_URI

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [10]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [11]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 931647533046-compute@developer.gserviceaccount.com


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [12]:
import google.cloud.aiplatform as aiplatform

#### Import TensorFlow

Import the TensorFlow package into your Python environment.

In [13]:
import tensorflow as tf

2022-04-01 07:27:22.459928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-01 07:27:22.459979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [14]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training.

Set the variable `TRAIN_GPU/TRAIN_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more [here](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) hardware accelerator support for your region

In [16]:
import os
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### Set pre-built containers

Set the pre-built Docker container image for training.

- Set the variable `TF` to the TensorFlow version of the container image. For example, `2-1` would be version 2.1, and `1-15` would be version 1.15. The following list shows some of the pre-built images available:


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).

In [17]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.1".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

Training: us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-1:latest AcceleratorType.NVIDIA_TESLA_K80 1


#### Set machine type

Next, set the machine type to use for training.

- Set the variable `TRAIN_COMPUTE` to configure  the compute resources for the VMs you will use for for training.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [18]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

Train machine type n1-standard-4


## Training with TensorBoard

Tensorboard provides the means to visualize your training in-real time and to visualize the results (metrics).

You can use Tensorboard in conjunction with local training, cloud training and with `Vertex AI Training`, which is referred to as `Vertex AI TensorBoard`

### Local training with TensorBoard

When using TensorBoard with TF.Keras training locally or on the cloud, you do the following steps:

1. Create a TensorBoard callback.
2. Pass the callback to the `fit()` method, specifying a local directory to write the tensorboard logs to.
3. Upload the log directory to TensorBoard.

Learn more about [TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard).

In [19]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten

LOG_DIR = "./logs"

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train / 255.0).astype(np.float32)
x_test = (x_test / 255.0).astype(np.float32)

model = Sequential(
    [
        Flatten(),
        Dense(512, activation="relu"),
        Dense(512, activation="relu"),
        Dense(10, activation="softmax"),
    ]
)

model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)


tensorboard = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR)

history = model.fit(x_train, y_train, epochs=4, batch_size=32, callbacks=[tensorboard])

! ls {LOG_DIR}/train/

11501568/11490434 [==============================] - 0s 0us/step


2022-04-01 07:31:48.914708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-01 07:31:48.914761: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-01 07:31:48.914795: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a4b0fa636d5c): /proc/driver/nvidia/version does not exist
2022-04-01 07:31:48.915543: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1839 - accuracy: 0.9432
Epoch 2/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0800 - accuracy: 0.9753
Epoch 3/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0557 - accuracy: 0.9821
Epoch 4/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0434 - accuracy: 0.9860
events.out.tfevents.1648798309.a4b0fa636d5c.1073.0.v2


### Uploading and sharing TensorBoard logs using `tensorboard dev`

You can upload your TensorBoard logs and share with others using `tensorboard dev` command. Once uploaded, a URL is returned to open up the TensorBoard instance in a brower for visualizing.

*Note:* Your TensorBoard instance is publicly visible.

Learn more about [What is TensorBoard.dev](https://tensorboard.dev/).

In [29]:
! tensorboard dev upload --logdir {LOG_DIR} \
  --name "Simple experiment with MNIST" \
  --description "Training results" \
  --one_shot<<{yes}

2022-04-01 08:31:02.377395: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-01 08:31:02.377454: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-01 08:31:04.557792: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-01 08:31:04.557840: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-01 08:31:04.557868: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a4b0fa636d5c): /proc/driver/nvidia/version does not exist

New experiment created. View your TensorBoard at: https://tenso

### Uploading TensorBoard logs using Vertex AI TensorBoard

You can upload your TensorBoard logs by first creating a TensorBoard instance and then using the `tb-gcp-uploader` command to upload the logs. Once uploaded, the command will return a URL for connecting to the TensorBoard instance via the browser.

Learn more about [TensorBoard overview](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview).

In [30]:
TENSORBOARD_DISPLAY_NAME = "example"
tensorboard = aiplatform.Tensorboard.create(display_name=TENSORBOARD_DISPLAY_NAME)
tensorboard_resource_name = tensorboard.gca_resource.name
print("TensorBoard resource name:", tensorboard_resource_name)

INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Creating Tensorboard
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Create Tensorboard backing LRO: projects/931647533046/locations/us-central1/tensorboards/2801889879108091904/operations/5486847315077496832
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Tensorboard created. Resource name: projects/931647533046/locations/us-central1/tensorboards/2801889879108091904
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:To use this Tensorboard in another session:
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:tb = aiplatform.Tensorboard('projects/931647533046/locations/us-central1/tensorboards/2801889879108091904')
TensorBoard resource name: projects/931647533046/locations/us-central1/tensorboards/2801889879108091904


Upload the logs with the command `tb-gcp-uploader`, with the following arguments:

--tensorboard_resource_name: The resource name of the TensorBoard instance.

--logdir: location of the TensorBoard logs.

--experiment_name: name for the experiment (training run).


Once uploaded retrieve the URL to the TensorBoard instance to visualize the training/results.

In [44]:
if 'EXPERIMENT_NAME' not in globals():
    EXPERIMENT_NAME='example-' + TIMESTAMP

output = !tb-gcp-uploader --tensorboard_resource_name={tensorboard_resource_name} \
  --logdir={LOG_DIR} \
  --experiment_name={EXPERIMENT_NAME} --one_shot=True

#print(output[5])
url = output[5].split(' ')[-1]

#print(url)

from IPython.display import display, HTML
display(HTML("<a href='" + url + "'>click here for TensorBoard instance</a>"))

E0401 08:54:50.335097634    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


### CustomTrainingJob training with Vertex AI TensorBoard

To use Vertex AI TensorBoard in conjunction with custom training with Vertex AI training, you make the following modifications:

**Python Training Script**:

1. Get the value of the environment variable `AIP_TENSORBOARD_LOG_DIR`. This is set by Vertex AI Training service.
2. Create a TensorBoard callback with the log_dir parameter set to the value of `AIP_TENSORBOARD_LOG_DIR`.

**CustomTrainingJob**:

1. On the `run()` method, add two additional parameters:

- `tensorboard`: The resource name of the Vertex TensorBoard instance.
- `service_account`: The service account with permissions to access Cloud Storage and Vertex TensorBoard.

### Examine the training package

#### Package layout

Before you start the training, you will look at how a Python package is assembled for a custom training job. When unarchived, the package contains the following directory/file layout.

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the Docker image.

The file `trainer/task.py` is the Python script for executing the custom training job. *Note*, when we referred to it in the worker pool specification, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).

#### Package Assembly

In the following cells, you will assemble the training package.

In [45]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: example image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

E0401 08:56:27.582974333    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:56:28.681905915    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:56:29.781195334    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:56:30.892969064    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:56:31.968876331    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:56:33.049758715    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:56:34.121872109    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:56:35.191005829    1073 fork_pos

#### Task.py contents

In the next cell, you write the contents of the custom training script task.py. In summary:

- Parse the command line arguments for the training configuration and hyperparameter settings.
 - Get the directory where to save the model artifacts from the command line (`--model_dir`), and if not specified, then from the environment variable `AIP_MODEL_DIR`.
 - Get the resource name of the Vertex AI TensorBoard instance, and if not specified, then from the environment variable `AIP_TENSORBOARD_LOG_DIR`.

- Load and preprocess the MNIST dataset.
- Build and compile a DNN model.
- Create a training callback to the TensorBoard instance.
- Train the model with the `fit()` method with the callback to the TensorBoard instance.

In [46]:
%%writefile custom/trainer/task.py
# Single Instance Training for MNIST

import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--batch_size', dest='batch_size',
                    default=20, type=int,
                    help='Batch size.')
parser.add_argument('--tensorboard_log_dir', dest='tensorboard_log_dir',
                    default=os.getenv('AIP_TENSORBOARD_LOG_DIR'), type=str,
                    help='Log directory for TensorBoard instance')
args = parser.parse_args()


def make_datasets():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = (x_train / 255.0).astype(np.float32)
    x_test  = (x_test  / 255.0).astype(np.float32)
    return (x_train, y_train), (x_test, y_test)


(x_train, y_train), _ = make_datasets()


def build_and_compile_model():
    model = Sequential([
        Flatten(),
        Dense(512, activation='relu'),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(args.lr),
        metrics=['accuracy'])
    return model

model = build_and_compile_model()


def train_model():
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=args.tensorboard_log_dir)

    history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.batch_size, callbacks=[tensorboard])
    return history

history = train_model()
model.save(args.model_dir)

Writing custom/trainer/task.py


#### Store training script on your Cloud Storage bucket

Next, you package the training folder into a compressed tar ball, and then store it in your Cloud Storage bucket.

In [47]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_example.tar.gz

E0401 08:58:01.012185802    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:58:02.231890122    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


custom/
custom/PKG-INFO
custom/README.md
custom/setup.py
custom/setup.cfg
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py


E0401 08:58:03.494794306    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0401 08:58:04.604318105    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.3 KiB/  1.3 KiB]                                                
Operation completed over 1 objects/1.3 KiB.                                      


### Create and run custom training job


To train a custom model, you perform two steps: 1) create a custom training job, and 2) run the job.

#### Create custom training job

A custom training job is created with the `CustomTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The training container image.
- `requirements`: Package requirements for the training container image (e.g., pandas).
- `script_path`: The relative path to the training script.

In [48]:
job = aiplatform.CustomTrainingJob(
    display_name="example_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### Prepare your command-line arguments

Now define the command-line arguments for your custom training container:

- `args`: The command-line arguments to pass to the executable that is set as the entry point into the container.
  - `--model-dir` : For our demonstrations, we use this command-line argument to specify where to store the model artifacts.
      - direct: You pass the Cloud Storage location as a command line argument to your training script (set variable `DIRECT = True`), or
      - indirect: The service passes the Cloud Storage location as the environment variable `AIP_MODEL_DIR` to your training script (set variable `DIRECT = False`). In this case, you tell the service the model artifact location in the job specification.
  - `"--epochs=" + EPOCHS`: The number of epochs for training.
  - `"--steps=" + STEPS`: The number of steps per epoch.

In [50]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### Run the custom training job

Next, you run the custom job to start the training job by invoking the method `run`, with the following parameters:

- `args`: The command-line arguments to pass to the training script.
- `replica_count`: The number of compute instances for training (replica_count = 1 is single node training).
- `machine_type`: The machine type for the compute instances.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `base_output_dir`: The Cloud Storage location to write the model artifacts to.
- `tensorboard`: The resource name of the Vertex AI TensorBoard instance.
- `service_account`: The service account with permissions to access Cloud Storage and Vertex AI TensorBoard.
- `sync`: Whether to block until completion of the job.

In [51]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        tensorboard=tensorboard_resource_name,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        tensorboard=tensorboard_resource_name,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

E0401 09:02:38.286802280    1073 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://vertex-ai-devaip-20220401072400/aiplatform-2022-04-01-09:02:38.747-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://vertex-ai-devaip-20220401072400/20220401072400 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4555624669306159104?project=931647533046
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/4555624669306159104 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4250734492970385408?project=931647533046
INFO:google.cloud.aiplatform.training_jobs:View tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/proj

### View your TensorBoard instance

To view a Vertex TensorBoard associated with a training job, navigate to the Training page in the Vertex AI section of the Google Cloud Console. Click the training job to view the Training Detail page, then click the Open TensorBoard button on the top of the page.

Alternatively, you can navigate to the Experiments tab and view the list of all experiments. Your experiment will have the same name as the training job.

### Delete the TensorBoard instance

Next, delete the TensorBoard instance.

In [52]:
tensorboard.delete()

INFO:google.cloud.aiplatform.base:Deleting Tensorboard : projects/931647533046/locations/us-central1/tensorboards/2801889879108091904
INFO:google.cloud.aiplatform.base:Delete Tensorboard  backing LRO: projects/931647533046/locations/us-central1/operations/5191298589531308032
INFO:google.cloud.aiplatform.base:Tensorboard deleted. . Resource name: projects/931647533046/locations/us-central1/tensorboards/2801889879108091904


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Model
- Custom Job
- Cloud Storage Bucket

In [54]:
# Delete the custom training job
job.delete()

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

INFO:google.cloud.aiplatform.base:Deleting CustomTrainingJob : projects/931647533046/locations/us-central1/trainingPipelines/4555624669306159104
INFO:google.cloud.aiplatform.base:Delete CustomTrainingJob  backing LRO: projects/931647533046/locations/us-central1/operations/7443098403216556032
INFO:google.cloud.aiplatform.base:CustomTrainingJob deleted. . Resource name: projects/931647533046/locations/us-central1/trainingPipelines/4555624669306159104
